In [3]:
import pygame, sys
import numpy as np
from keras.models import load_model
import cv2
from pygame.locals import *

In [ ]:
# Set constants
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
WINDOWSIZEX = 640
WINDOWSIZEY = 480
BOUNDRYINC = 5
IMAGESAVE = False

# Load the model
MODEL = load_model('Trained-model.h5')

# Flag for prediction
PREDICT = True

# Labels for prediction
LABELS = {
    0: "zero",
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine",
}

# Initialize pygame
pygame.init()

# Set up display
DISPLAYSURF = pygame.display.set_mode((WINDOWSIZEX, WINDOWSIZEY))
pygame.display.set_caption('Digit Board')

# Flag for writing
iswriting = False

# Lists to store coordinates
number_xcord = []
number_ycord = []

# Counter for images
image_cnt = 1


# Event loop
while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

        if event.type == MOUSEMOTION and iswriting:
            xcord, ycord = event.pos
            pygame.draw.circle(DISPLAYSURF, WHITE, (xcord, ycord), 4, 0)
            number_xcord.append(xcord)
            number_ycord.append(ycord)

        if event.type == MOUSEBUTTONDOWN:
            iswriting = True

        if event.type == MOUSEBUTTONUP:
            iswriting = False
            number_xcord = sorted(number_xcord)
            number_ycord = sorted(number_ycord)
            
            # Get bounding box
            rect_min_x, rect_max_x = max(number_xcord[0] - BOUNDRYINC, 0), min(WINDOWSIZEX, number_xcord[-1] + BOUNDRYINC)
            rect_min_Y, rect_max_Y = max(number_ycord[0] - BOUNDRYINC, 0), min(number_ycord[-1] + BOUNDRYINC, WINDOWSIZEY)
            
            number_xcord = []
            number_ycord = []

            # Capture image and process
            img_arr = np.array(pygame.PixelArray(DISPLAYSURF))[rect_min_x:rect_max_x, rect_min_Y:rect_max_Y].T.astype(np.float32)

            if IMAGESAVE:
                cv2.imwrite(f'image{image_cnt}.png', img_arr)
                image_cnt += 1

            if PREDICT:
                # Resize and preprocess image
                img_arr = cv2.resize(img_arr, (28, 28))

                # Reshape the image array and normalize
                img_arr = img_arr.reshape(1, 28, 28, 1) / 255.0
                
                # Predict
                prediction = MODEL.predict(img_arr)
                label = str(LABELS[np.argmax(prediction)])

                # Display label
                font = pygame.font.Font(None, 36)
                text_surface = font.render(label, True, RED)
                text_rect = text_surface.get_rect()
                text_rect.topleft = (rect_min_x, rect_max_Y)
                DISPLAYSURF.blit(text_surface, text_rect)

        if event.type == KEYDOWN:
            if event.unicode == 'n':
                DISPLAYSURF.fill(BLACK)

    pygame.display.update()

1/1 [==============================] - 0s 40ms/step
